In [69]:
# import libraries
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import nltk
import string
lem = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english')) 
exclude = set(string.punctuation)
import string
import warnings; warnings.simplefilter('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/uplarauser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/uplarauser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/uplarauser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Content (Metadata) Based Recommender

In [3]:
df=pd.read_csv("reviews.csv")

In [4]:
df.head()

,place,reviews
0,Lal Tibba,The best time to visit this attraction is earl...
1,Lal Tibba,"Disappointing, considering the hype around it...."
2,Lal Tibba,Nice location since it was clear sky we could ...
3,Lal Tibba,Owasm place during the ⛅ sunrise & sunset. A c...
4,Lal Tibba,Its just a 2 Story structure with a old age bi...


In [55]:
a=pd.DataFrame(df.groupby("place")["reviews"].apply(list)).reset_index()

In [56]:
a.head()

,place,reviews
0,Anasakti Ashram,"[A great place to visit, enjoy the serenity an..."
1,Baijnath Temple,[This a cluster of stone temples located in Ka...
2,Bhalu Gaad Waterfalls,[It's an amazing waterfall which can bring all...
3,Bhimtal Island Aquarium,"[During my visit, i had a morning walk from ho..."
4,Bhimtal Lake,[The hotel is located in a wonderful destinati...


In [57]:
a["reviews"]=a["reviews"].apply(lambda x: ' '.join(x))

In [58]:
a.head()

,place,reviews
0,Anasakti Ashram,"A great place to visit, enjoy the serenity and..."
1,Baijnath Temple,This a cluster of stone temples located in Kas...
2,Bhalu Gaad Waterfalls,It's an amazing waterfall which can bring all ...
3,Bhimtal Island Aquarium,"During my visit, i had a morning walk from hot..."
4,Bhimtal Lake,The hotel is located in a wonderful destinatio...


In [59]:
def filter_keywords(doc):
    doc=doc.lower()
    stop_free = " ".join([i for i in doc.split() if i not in stop_words])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    word_tokens = word_tokenize(punc_free)
    filtered_sentence = [(lem.lemmatize(w, "v")) for w in word_tokens]
    return filtered_sentence

In [60]:
a['reviews'] = a['reviews'].apply(filter_keywords)
a["reviews"]=a["reviews"].apply(lambda x: ' '.join(x))

In [62]:
a.head()

,place,reviews
0,Anasakti Ashram,great place visit enjoy serenity calmness alon...
1,Baijnath Temple,cluster stone temples locate kasauni 2 hours a...
2,Bhalu Gaad Waterfalls,amaze waterfall bring sense alive whether roar...
3,Bhimtal Island Aquarium,visit morning walk hotel enter road reach isla...
4,Bhimtal Lake,hotel locate wonderful destination beside lake...


In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
# count_matrix = count.fit_transform(a['reviews'])
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(a['reviews'])

## Cosine Similarity
I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two places. Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score.

In [64]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

We now have a pairwise cosine similarity matrix for all the places in our dataset. The next step is to write a function that returns the most similar products based on the cosine similarity score.

In [65]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    product_indices = [i[0] for i in sim_scores]
    return titles.iloc[product_indices]

In [66]:
a = a.reset_index()
titles = a['place']
indices = pd.Series(a.index, index=a['place'])

Let us now try and get the top recommendations for a few places.

In [67]:
get_recommendations("Anasakti Ashram").head(5)

37                    Lakshmi Ashram
63                  Sivananda Ashram
39             Ma Anandamayee Ashram
64                   Snow View Point
41    Mukteshwar Inspection Bungalow
Name: place, dtype: object

In [68]:
get_recommendations("Bhimtal Lake").head(5)

45              Nainital Lake
75               Victoria Dam
5             Bhulla Tal Lake
3     Bhimtal Island Aquarium
25         Goddess Naina Devi
Name: place, dtype: object